In [1]:
import csv
from bs4 import BeautifulSoup

In [2]:
# for Firefox and Chrome
!pip install selenium
from selenium import webdriver

In [3]:
# start up the chrome webdriver
driver=webdriver.Chrome()

In [4]:
url='https://www.amazon.in/'
driver.get(url)

In [5]:
# when you write the search term in the search bar you get the search term embedded in the url
# We can use this pattern to create a fucntion that will build the necessary url for our driver to retrieve
# this will be very efficient in the long term and less likely to encounter error

In [29]:
def get_url(search_term):
    # Generate the url from the searched term
    template='https://www.amazon.in/s?k={}&ref=nb_sb_noss_1'
    search_term=search_term.replace(' ','+')
    return template.format(search_term)

In [30]:
url=get_url('ultrawide monitor')
print(url)

https://www.amazon.in/s?k=ultrawide+monitor&ref=nb_sb_noss_1


In [95]:
driver.get(url)   # with this line, the url is searched in the browser and the given page will open

## Extract the collection

In [105]:
# Creating soup object to parse the html content
soup=BeautifulSoup(driver.page_source, 'html.parser')  # default html.parser to parse the page

In [106]:
# now we have to find unique elements in our soup object

In [107]:
type(soup)

bs4.BeautifulSoup

In [108]:
# Our next object is to inpect the page and find the tag that encloses any unique record
# In this case the div tag enclose each single record

In [109]:
results=soup.find_all('div', {'data-component-type':'s-search-result'})   # didn't understood this one well so please refer to the actual video https://www.youtube.com/watch?v=_AeudsbKYG8

In [110]:
len(results)  # the number in results also include the ads present in the webpage

22

## Prototype the record 

In [111]:
#one protype will be to get others

In [115]:
item=results[0]
#item-> one unique searched record

In [38]:
# using tree navigation technique to get down that tree of tags
atag=item.h2.a

In [39]:
atag.text

'Gigabyte G27F 27" 144Hz 1080P Gaming Monitor, 1920 x 1080 IPS Display, 1ms (MPRT) Response Time, 95% DCI-P3, FreeSync Premium '

In [40]:
description=atag.text.strip()  # strip() method to remove the extra space at the edges

In [41]:
description

'Gigabyte G27F 27" 144Hz 1080P Gaming Monitor, 1920 x 1080 IPS Display, 1ms (MPRT) Response Time, 95% DCI-P3, FreeSync Premium'

In [43]:
atag.get('href')  # getting the url for the atag; its not the full url

'/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A00231513511X0MK6OS7S&url=%2FGigabyte-G27F-Monitor-Response-FreeSync%2Fdp%2FB08WHHZBF8%2Fref%3Dsr_1_1_sspa%3Fdchild%3D1%26keywords%3Dultrawide%2Bmonitor%26qid%3D1632946330%26sr%3D8-1-spons%26psc%3D1&qualifier=1632946330&id=2783049559006316&widgetName=sp_atf'

In [126]:
url='https://www.amazon.in'+atag.get('href')
url

'https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A00231513511X0MK6OS7S&url=%2FGigabyte-G27F-Monitor-Response-FreeSync%2Fdp%2FB08WHHZBF8%2Fref%3Dsr_1_1_sspa%3Fdchild%3D1%26keywords%3Dultrawide%2Bmonitor%26qid%3D1632946330%26sr%3D8-1-spons%26psc%3D1&qualifier=1632946330&id=2783049559006316&widgetName=sp_atf'

In [127]:
# next item we need is price
price_parent=item.find('span', 'a-price')
price_parent

<span class="a-price" data-a-color="price" data-a-size="l"><span class="a-offscreen">₹12,990</span><span aria-hidden="true"><span class="a-price-symbol">₹</span><span class="a-price-whole">12,990</span></span></span>

In [52]:
price=price_parent.find('span', 'a-offscreen').text
price

'₹20,100'

In [58]:
# getting the rating
rating=item.i.text
rating

'4.1 out of 5 stars'

In [63]:
# for getting the reviews we'll use the dir and class props.
reviews_count=item.find('span', {'class':'a-size-base'}).text
reviews_count

'41'

## Generalize the pattern

In [74]:
def extract_pattern(item):
    # Extract and return data from a single record
    
    #Description and url
    atag=item.h2.a
    description=atag.text.strip()
    url='https://www.amazon.in'+atag.get('href')

    # Price
    price_parent=item.find('span', 'a-price')
    price=price_parent.find('span','a-offscreen').text

    
    # rank and rating
    rating=item.i.text
    reviews_count=item.find('span', {'class':'a-size-base'}).text
    
    # creating a tuple to store all these items
    result=(description, price, rating, reviews_count, url)
    return result

In [129]:
# Now applying this pattern to all the records on this page
records=[]  # to store records
results=soup.find_all('div', {'data-component-type':'s-search-result'})

results[0]

<div class="s-result-item s-asin sg-col-0-of-12 sg-col-16-of-20 AdHolder sg-col sg-col-12-of-16" data-asin="B07DP7RYXV" data-cel-widget="search_result_1" data-component-id="54" data-component-type="s-search-result" data-index="1" data-uuid="2110804f-914c-4f1b-bcce-11653d365184"><div class="sg-col-inner">
<span cel_widget_id="MAIN-SEARCH_RESULTS-1" class="celwidget slot=MAIN template=SEARCH_RESULTS widgetId=search-results_1" data-cel-widget="MAIN-SEARCH_RESULTS-1" data-csa-c-id="5izx2q-uv1haa-seqalx-dl4o0e">
<div class="rush-component" data-component-id="55" data-component-props='{"percentageShownToFire":"50","batchable":true,"requiredElementSelector":".s-image:visible","url":"https://www.amazon.in/gp/sponsored-products/logging/log-action.html?qualifier=1633030895&amp;id=589402287456840&amp;widgetName=sp_atf&amp;adId=20077892312602&amp;eventType=1&amp;adIndex=0"}' data-component-type="s-impression-logger">
<div class="rush-component" data-component-id="56" data-component-type="sp-sponso

In [71]:
# the above error is because our model seeks for all these 5 atrributes for every record on the page
# but many of the records don't have all 5 attributes
#Hence will do some error handling

In [75]:
# If there is no price i.e., that item is not available hence no prblem
# But if there is no rating or reviews I stilll want to see or but that product

### Hence we will add error handling inside extract_pattern fucntion

In [76]:
def extract_record(item):
    # Extract and return data from a single record

    #Description and url
    atag=item.h2.a
    description=atag.text.strip()
    url='https://www.amazon.in'+atag.get('href')

    try:
        # Price
        price_parent=item.find('span', 'a-price')
        price=price_parent.find('span','a-offscreen').text
    except AttributeError:
        return   # return here means i don't care about it; let it be
    try:
        # rank and rating
        rating=item.i.text
        reviews_count=item.find('span', {'class':'a-size-base'}).text
    except AttributeError:   # if there is an attribute error return below given values
        rating=''            
        reviews_count=''
        
    # creating a tuple to store all these items
    result=(description, price, rating, reviews_count, url)
        
    return result

In [79]:
# Now applying this pattern to all the records on this page
records=[]  # to store records
results=soup.find_all('div', {'data-component-type':'s-search-result'})

for item in results:
    record=extract_record(item)
    if record:                 # if record has something in it
        records.append(record)

In [85]:
records[0]

('Gigabyte G27F 27" 144Hz 1080P Gaming Monitor, 1920 x 1080 IPS Display, 1ms (MPRT) Response Time, 95% DCI-P3, FreeSync Premium',
 '₹20,100',
 '4.1 out of 5 stars',
 '41',
 'https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A00231513511X0MK6OS7S&url=%2FGigabyte-G27F-Monitor-Response-FreeSync%2Fdp%2FB08WHHZBF8%2Fref%3Dsr_1_1_sspa%3Fdchild%3D1%26keywords%3Dultrawide%2Bmonitor%26qid%3D1632946330%26sr%3D8-1-spons%26psc%3D1&qualifier=1632946330&id=2783049559006316&widgetName=sp_atf')

In [84]:
len(records)

21

In [91]:
for row in records:
    print(row[1])

₹20,100
₹41,159
₹18,899
₹17,999
₹29,392
₹29,299
₹15,119
₹14,999
₹53,359
₹13,999
₹69,999
₹27,900
₹49,999
₹37,999
₹16,683
₹53,320
₹19,799
₹20,327
₹31,990
₹54,989
₹11,499


In [137]:
# URL with page number embedded
def get_url(search_term):
    # Generate the url from the searched term
    template='https://www.amazon.in/s?k={}&ref=nb_sb_noss_1'
    search_term=search_term.replace(' ','+')
    
    # add search term query to url
    url=tempate.format(search_term)
    
    # add page query pageholder
    url+='&page={}'
    
    return template.format(search_term)

## Now putting it all together

In [130]:
import csv 
from bs4 import BeautifulSoup
from selenium import webdriver

# url with page number embedded
def get_url(search_term):
    # Generate the url from the searched term
    template='https://www.amazon.in/s?k={}&ref=nb_sb_noss_1'
    search_term=search_term.replace(' ','+')
    
    # add search term query to url
    url=template.format(search_term)
    
    # add page query pageholder
    url+='&page{}'
    
    return url

def extract_record(item):
    # Extract and return data from a single record
    
    #Description and url
    atag=item.h2.a     # for moving down the tag tree to the href which contains url
    description=atag.text.strip()    # removing spaces at edges
    url='https://www.amazon.in'+atag.get('href')
    
    try:    
        # Price
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span','a-offscreen').text
    except AttributeError:
        return   # return here means if i don't get the price whole record is skipped
    
    try:
        # rank and rating
        rating=item.i.text
        reviews_count=item.find('span', {'class':'a-size-base'}).text
    except AttributeError:   # if there is an attribute error return below given values
        rating=''            
        reviews_count=''
        
    # creating a tuple to store all these items
    result=(description, price, rating, reviews_count, url)
        
    return result

# this main function is going to give the instructions that we want
def main(search_term):  # accepting argument of the search term; then going to run the search term and 
    
    # startup the webdriver
    driver=webdriver.Chrome()
    
    record=[]
    url=get_url(search_term)     # getting the url using the search term
    
    #  now going to iterate through 20 pages
    for page in range(1,21):
        # opening the webpage via url
        driver.get(url.format(page))    # opening the specified page
        soup=BeautifulSoup(driver.page_source, 'html.parser')   # soup object to parse the html code  
        results=soup.find_all('div', {'data-component-type':'s-search-result'})  # getting all the html content on the page
        
        # then for each of the results, gonna extract the results data and append it to our record list
        for item in results:
            record=extract_record(item)
            if record:
                records.append(record)
        
    driver.close()   # closing the driver
    
    # save data to our csv file
    with open('amazon.csv','w', newline='', encoding='utf-8') as f:    #unable to understand newline; encoding is the name of the encoding used to encode/decode the text
        writer=csv.writer(f)   # This function in csv module returns a writer object that converts data into a delimited string and stores in a file object.
        writer.writerow(['description', 'price', 'rating', 'reviewCount', 'url'])  #This function writes items in an iterable (list, tuple or string) ,separating them nby comma character.
        writer.writerows(records)  #This function takes a list of iterables as parameter and writes each item as a comma separated line of items in the file.
    

In [131]:
main('ultrawide monitor')